In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import  ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain import PromptTemplate, FewShotPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from operator import itemgetter
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

# 예시 데이터 정의
examples = [
    {
        "input": "타이타닉",
        "output": "🚢💑❄️"
    },
    {
        "input": "매트릭스",
        "output": "🕶️💊🤖"
    },
    {
        "input": "쥬라기 공원",
        "output": "🦖🏃🌴"
    },
    {
        "input": "해리 포터",
        "output": "⚡🧙‍♂️🪄"
    }
]

# Few-shot 프롬프트 템플릿 생성
few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=ChatPromptTemplate.from_messages([
        ("human", "{input}"),
        ("assistant", "{output}")
    ]),
)

# 메인 프롬프트 템플릿
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 영화 제목을 받아서 그 영화를 가장 잘 표현하는 3개의 이모지로 응답하는 assistant입니다. 반드시 3개의 이모지만 사용해야 합니다."),
    few_shot_prompt,
    ("human", "{input}"),
])

# 메모리 설정
memory = ConversationBufferMemory(
    return_messages=True,
    memory_key="chat_history",
    input_key="input"
)

# LLM 설정
llm = ChatOpenAI(
    temperature=0.7,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
    model="gpt-3.5-turbo",
)

# 체인 구성
chain = (
    RunnablePassthrough.assign(chat_history=memory.load_memory_variables) 
    | {"input": lambda x: x["input"], "chat_history": lambda x: x["chat_history"]}
    | prompt 
    | llm
    | StrOutputParser()
)

# 실행 함수
def run_chain(user_input):
    result = chain.invoke({"input": user_input})
    memory.save_context({"input": user_input}, {"output": result})
    return result

# 첫 번째 영화 테스트
run_chain("인셉션")

# 두 번째 영화 테스트
run_chain("아바타")

# 메모리 확인
memory.load_memory_variables({})["chat_history"]


💭🕰️🔁인셉션 -> 💭🕰️🔁
🌿🌊🌌아바타 -> 🌿🌊🌌


[HumanMessage(content='인셉션', additional_kwargs={}, response_metadata={}),
 AIMessage(content='💭🕰️🔁', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='아바타', additional_kwargs={}, response_metadata={}),
 AIMessage(content='🌿🌊🌌', additional_kwargs={}, response_metadata={})]